<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Broyden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True)
from scipy.optimize import root, minimize, NonlinearConstraint
eps=1e-12

In [2]:
def func(x):
    return jnp.array([jnp.sin(x[0])  + 0.5 * (x[0] - x[1])**3 - 1.0,
            0.5 * (x[1] - x[0])**3 + x[1]])

In [3]:
# Broyden update with no Sherman-Morrison
x = jnp.zeros(2)
J = jax.jacobian(func)(x)

f = func(x)

for i in range(20):
  xp = jnp.linalg.solve(J, -f) + x
  dx = xp - x
  fp = func(xp)
  f= fp
  x= xp
  print(x,fp)
  if jnp.linalg.norm(fp) < 1e-12:
    break

  J = J + jnp.outer(fp,dx)/jnp.linalg.norm(dx)**2

/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


[1. 0.] [ 0.34147098 -0.5       ]
[0.74545034 0.37272517] [-0.29580697  0.34683492]
[0.87452582 0.23424262] [-0.10151446  0.10299655]
[0.94426608 0.18266999] [ 0.03094058 -0.03820376]
[0.92886637 0.19874589] [-0.00445347  0.00414106]
[0.931058   0.19761267] [-0.00047246  0.00033713]
[0.93134054 0.1975642 ] [-3.66025399e-05  2.14376618e-05]
[0.93136529 0.19756394] [-1.63573041e-06  9.88074612e-07]
[0.93136644 0.19756391] [ 9.80559234e-09 -5.98158895e-09]
[0.93136643 0.19756391] [-1.81729076e-11  1.10534082e-11]
[0.93136643 0.19756391] [-1.01030295e-14  6.16173779e-15]


In [5]:
def broyden(func, x, J=None, max_iter=100, verbose=0):
  J = jax.jacobian(func)(x) if J is None else J(x)
  Jinv = jnp.linalg.inv(J)
  f = func(x)

  for i in range(max_iter):
    xp = x - Jinv @ f
    dx = xp - x
    fp = func(xp)
    f= fp
    x= xp
    if verbose>0:
      print(x, f)
    if jnp.linalg.norm(fp) < 1e-12:
      break

    u = jnp.expand_dims(fp,1)
    v = jnp.expand_dims(dx,1)/jnp.linalg.norm(dx)**2
    Jinv = Jinv - Jinv @ u @ v.T @ Jinv / (1 + v.T @ Jinv @ u)  #Sherman-Morrison
  return x, f

broyden(func, jnp.zeros(2))

(DeviceArray([0.93136643, 0.19756391], dtype=float64),
 DeviceArray([-1.01030295e-14,  6.16173779e-15], dtype=float64))

In [6]:
prng = jax.random.PRNGKey(1234)
J = jax.random.uniform(prng, (3,3))
f = jax.random.uniform(prng, (3,1))
fp = jax.random.uniform(prng, (3,1))
dx = jnp.linalg.solve(J,-f)

def constraint(x):
    Jp = x.reshape((3,3))
    return jnp.squeeze(Jp @ dx)

nlc = NonlinearConstraint(constraint,jnp.squeeze(fp-f),jnp.squeeze(fp-f), jac=jax.jacobian(constraint))

def func(x):
  Jp = x.reshape((3,3))
  return jnp.linalg.norm(Jp-J)

res = minimize(func, jnp.zeros(9), method='SLSQP', jac=jax.jacobian(func), constraints=(nlc))
print(res.x.reshape((3,3))) #via constrained minimization
print(J + fp@dx.T/jnp.linalg.norm(dx)**2) #via Broyden update

[[0.81624494 0.71880643 0.15777668]
 [0.22143383 0.01955389 0.22160245]
 [0.7486602  0.24306155 0.56889726]]
[[0.8163133  0.71886663 0.15778989]
 [0.22145238 0.01955553 0.221621  ]
 [0.7487229  0.24308191 0.56894491]]


In [7]:
# Accomodate box bounds on variables

def broyden2(func, x, J=None, max_iter=100, verbose=0, xmax=jnp.inf, xmin=-jnp.inf):
  Jf = jax.jacobian(func) if J is None else J
  J = Jf(x)
  Jinv = jnp.linalg.inv(J)
  f = func(x)

  for i in range(max_iter):
    dx = - Jinv @ f

    alpha_max_limits = jnp.min(jnp.where(x + dx > xmax, (xmax - x) / (dx), 1))
    alpha_min_limits = jnp.min(jnp.where(x + dx < xmin, (xmin - x) / (dx), 1))
    alpha = min(alpha_max_limits, alpha_min_limits)

    print(alpha)
    dx = alpha*dx
    xp = x + dx
    fp = func(xp)
    f= fp
    x= xp
    if verbose>0:
      print(x, f)
    if jnp.linalg.norm(fp) < 1e-12:
      break

    u = jnp.expand_dims(fp,1)
    v = jnp.expand_dims(dx,1)/jnp.linalg.norm(dx)**2
    Jinv = Jinv - Jinv @ u @ v.T @ Jinv / (1 + v.T @ Jinv @ u)  #Sherman-Morrison
  return x, f

In [8]:
def func2(x):
    return jnp.array([jnp.sin(x[0])  + 0.5 * (x[0] - x[1])**3 - 0.01*jnp.sqrt(x[1]-0.1) - 1.0,
            0.5 * (x[1] - x[0])**3 + x[1] + 0.001*jnp.sqrt(1.-x[0])])

In [9]:
broyden(func2, 0.95*jnp.ones(2),verbose=1, max_iter=20)

[1.27776387e+00 5.09295507e-04] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]
[nan nan] [nan nan]


(DeviceArray([nan, nan], dtype=float64),
 DeviceArray([nan, nan], dtype=float64))

In [10]:
broyden2(func2, 0.95*jnp.ones(2), verbose=1, max_iter=20, xmin=jnp.array([-jnp.inf, 0.1]), xmax = jnp.array([1., jnp.inf]))

0.1525488433491314
[1.         0.80515629] [-0.16322784  0.80145776]
1.0
[0.93959614 0.98225636] [-0.20211182  0.98254095]
0.19327261837160434
[1.         0.79656208] [-0.1626652   0.79235224]
1.0
[0.92070903 1.0408568 ] [-0.21453601  1.04200559]
0.24205077931482566
[1.         0.79406184] [-0.16249308  0.78969486]
1.0
[0.88356639 1.15670184] [-0.24746165  1.16723142]
0.3484361198064109
[1.         0.76610502] [-0.16029271  0.75970719]
1.0
[0.75065498 1.6076198 ] [-0.6448333   1.92279178]
1.0
[0.9274454  0.20208157] [-0.01227885  0.01152487]
1.0
[0.94127706 0.19437996] [ 0.01356875 -0.01370796]
1.0
[0.93420749 0.19877441] [-0.00013078  0.00014708]
1.0
[0.93426884 0.1987212 ] [-5.09566974e-07  7.82997886e-07]
1.0
[0.93426899 0.19872083] [ 1.0061536e-08 -8.9609632e-09]
1.0
[0.93426899 0.19872083] [ 2.1699087e-10 -1.9291255e-10]
1.0
[0.93426899 0.19872083] [ 1.08801856e-14 -9.52585450e-15]


(DeviceArray([0.93426899, 0.19872083], dtype=float64),
 DeviceArray([ 1.08801856e-14, -9.52585450e-15], dtype=float64))

In [38]:
# Limit step size to ensure a decrease in norm

def broyden3(func, x, J=None, max_iter=100, verbose=0, xmax=jnp.inf, xmin=-jnp.inf):
  Jf = jax.jacobian(func) if J is None else J
  J = Jf(x)
  Jinv = jnp.linalg.inv(J)
  f = func(x)

  for i in range(max_iter):
    print(f"\nIter: {i}")
    dx = - Jinv @ f

    alpha_max_limits = jnp.min(jnp.where(x + dx > xmax, (xmax - x) / (dx), 1))
    alpha_min_limits = jnp.min(jnp.where(x + dx < xmin, (xmin - x) / (dx), 1))
    alpha = min(alpha_max_limits, alpha_min_limits)

    while alpha > 0.01:
      dx_try = alpha*dx
      xp = x + dx_try
      fp = func(xp)
      dnorm = jnp.linalg.norm(fp)-jnp.linalg.norm(f)
      print(f"Alpha {alpha}   dnorm {dnorm}  dx_try {dx_try}   f {f}    fp {fp}")
      if dnorm > 0:
        alpha *= 0.5
      else:
        break
    if alpha <= 0.01:
      print("reevaluate J")
      Jinv = jnp.linalg.inv(Jf(x))
      continue

    dx=dx_try
    f= fp
    x= xp
    if verbose>0:
      print(x, f)
    if jnp.linalg.norm(fp) < 1e-12:
      break

    u = jnp.expand_dims(fp,1)
    v = jnp.expand_dims(dx,1)/jnp.linalg.norm(dx)**2
    Jinv = Jinv - Jinv @ u @ v.T @ Jinv / (1 + v.T @ Jinv @ u)  #Sherman-Morrison
  return x, f

In [39]:
broyden3(func2, 0.95*jnp.ones(2), verbose=1, max_iter=20, xmin=jnp.array([-jnp.inf, 0.1+eps]), xmax = jnp.array([1.-eps, jnp.inf]))


Iter: 0
Alpha 0.1525488433460805   dnorm -0.1522769994586285  dx_try [ 0.05       -0.14484371]   f [-0.19580404  0.95022361]    fp [-0.16322784  0.80145776]
[1.         0.80515629] [-0.16322784  0.80145776]

Iter: 1
Alpha 1.0   dnorm 0.18520243065511433  dx_try [-0.06040385  0.17710007]   f [-0.16322784  0.80145776]    fp [-0.20211182  0.98254095]
Alpha 0.5   dnorm 0.09447817784862322  dx_try [-0.03020193  0.08855003]   f [-0.16322784  0.80145776]    fp [-0.18391719  0.89365983]
Alpha 0.25   dnorm 0.04810126644389001  dx_try [-0.01510096  0.04427502]   f [-0.16322784  0.80145776]    fp [-0.17419768  0.84831118]
Alpha 0.125   dnorm 0.024328390472125316  dx_try [-0.00755048  0.02213751]   f [-0.16322784  0.80145776]    fp [-0.16890823  0.82512827]
Alpha 0.0625   dnorm 0.012250770451923909  dx_try [-0.00377524  0.01106875]   f [-0.16322784  0.80145776]    fp [-0.1661218  0.8133705]
Alpha 0.03125   dnorm 0.006155804409375354  dx_try [-0.00188762  0.00553438]   f [-0.16322784  0.80145776] 

(DeviceArray([0.93426899, 0.19872083], dtype=float64),
 DeviceArray([ 8.65973959e-15, -4.47477342e-15], dtype=float64))

In [ ]:
# 